<a href="https://colab.research.google.com/github/jjyjung/ai/blob/gh-pages/tf_kera_CNN_CU200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')   # google drive를 google colab에 연결. 처음 실행 시, 인증 필요 

Mounted at /content/gdrive


In [ ]:
!unzip '/content/gdrive/My Drive/Colab Notebooks/ai/CUB200.zip' -d '/content/gdrive/My Drive/Colab Notebooks/ai/CUB200'

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
import os

dir = "/content/gdrive/My Drive/Colab Notebooks/ai/"
train_folder = dir + 'CUB20/train'
test_folder = dir + 'CUB20/test'

class_reduce=0.1 # 부류 수 줄여서 데이터양 줄임(속도와 메모리 효율을 위해)
no_class=int(len(os.listdir(train_folder))*class_reduce) # 부류 개수

x_train,y_train=[],[]
for i,class_name in enumerate(os.listdir(train_folder)):
    if i<no_class: # 13~14행이 지정한 부류만 사용
        for fname in os.listdir(train_folder+'/'+class_name):
            img=image.load_img(train_folder+'/'+class_name+'/'+fname,target_size=(224,224))
            if len(img.getbands())!=3:
                print("주의: 유효하지 않은 영상 발생",class_name,fname)
                continue
            x=image.img_to_array(img)
            x=preprocess_input(x)
            x_train.append(x)
            y_train.append(i)

x_test,y_test=[],[]
for i,class_name in enumerate(os.listdir(test_folder)):
    if i<no_class: # 13~14행이 지정한 부류만 사용
        for fname in os.listdir(test_folder+'/'+class_name):
            img=image.load_img(test_folder+'/'+class_name+'/'+fname,target_size=(224,224))
            if len(img.getbands())!=3:
                print("주의: 유효하지 않은 영상 발생",class_name,fname)
                continue
            x=image.img_to_array(img)
            x=preprocess_input(x)
            x_test.append(x)
            y_test.append(i)

x_train=np.asarray(x_train)
y_train=np.asarray(y_train)
x_test=np.asarray(x_test)
y_test=np.asarray(y_test)
y_train=tf.keras.utils.to_categorical(y_train,no_class)
y_test=tf.keras.utils.to_categorical(y_test,no_class)

base_model=ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
cnn=Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024,activation='relu'))
cnn.add(Dense(no_class,activation='softmax'))

cnn.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
hist=cnn.fit(x_train,y_train,batch_size=16,epochs=10,validation_data=(x_test,y_test),verbose=1)

res=cnn.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)

94781440/94765736 [==============================] - 1s 0us/step
Epoch 1/10
38/38 [==============================] - 60s 591ms/step - loss: 2.7399 - accuracy: 0.3450 - val_loss: 8.2893 - val_accuracy: 0.0117
Epoch 2/10
38/38 [==============================] - 19s 510ms/step - loss: 0.0228 - accuracy: 0.9950 - val_loss: 9.3647 - val_accuracy: 0.0214
Epoch 3/10
38/38 [==============================] - 20s 542ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 9.2411 - val_accuracy: 0.0155
Epoch 4/10
38/38 [==============================] - 19s 509ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 9.0796 - val_accuracy: 0.0097
Epoch 5/10
38/38 [==============================] - 19s 509ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 9.0452 - val_accuracy: 0.0117
Epoch 6/10
38/38 [==============================] - 19s 509ms/step - loss: 8.0682e-04 - accuracy: 1.0000 - val_loss: 8.9971 - val_accuracy: 0.0136
Epoch 7/10
38/38 [==============================] - 20s 542ms/step - loss